## 第11章 时间序列
在多个时间点观察或测量到的任何事物都可以形成一段时间序列。<br />
很多时间序列是固定频率的，也就是说，数据点是根据某种规律<br />
定期出现的（比如每15秒、每5分钟、每月出现一次）。<br />
时间序列也可以是不定期的，没有固定的时间单位或单位之间的偏移量。
时间戳（timestamp），特定的时刻。<br />
固定时期（period），如2007年1月或2010年全年。<br />
时间间隔（interval），由起始和结束时间戳表示。时期（period）<br/>
可以被看做间隔（interval）的特例。<br />
实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。<br />
例如，从放入烤箱时起，每秒钟饼干的直径。
pandas提供了许多内置的时间序列处理工具和数据算法。因此，你<br/>
可以高效处理非常大的时间序列，轻松地进行切片/切块、聚合、对<br/>
定期/不定期的时间序列进行重采样等。有些工具特别适合金融和经<br/>
济应用，你当然也可以用它们来分析服务器日志数据。

In [19]:
import numpy as np
import pandas as pd

### 11.1 日期和时间数据类型及工具
Python标准库包含用于日期（date）和时间（time）数据的数据<br/>
类型，而且还有日历方面的功能。我们主要会用到datetime、time<br/>
以及calendar模块。<br/>
datetime.datetime（也可以简写为datetime）是用得最多的数据类型

In [2]:
from datetime import datetime

now = datetime.now()

print(now)

now.year, now.month, now.day

2019-05-21 17:33:10.716422


(2019, 5, 21)

In [5]:
#datetime以毫秒形式存储日期和时间。
#timedelta表示两个datetime对象之间的时间差
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)

print(delta)

print(delta.days)

print(delta.seconds)

926 days, 15:45:00
926
56700


In [7]:
#可以给datetime对象加上（或减去）一个或多个timedelta，
#这样会产生一个新对象
from datetime import timedelta

start = datetime(2011, 1, 7)

print(start + timedelta(12))

start - 2 * timedelta(12)#12 是day

2011-01-19 00:00:00


datetime.datetime(2010, 12, 14, 0, 0)

#### 字符串和datatime的相互转换

In [12]:
#利用str或strftime方法（传入一个格式化字符串），
#datetime对象和pandas的Timestamp对象（稍后就会介绍）
#可以被格式化为字符串
stamp = datetime(2011, 1, 3)

print(str(stamp))

stamp.strftime('%Y-%m-%d')

2011-01-03 00:00:00


'2011-01-03'

In [14]:
#datetime.strptime可以用这些格式化编码将字符串转换为日期
value = '2011-01-03'

print(datetime.strptime(value,'%Y-%m-%d'))

datestrs = ['7/6/2011', '8/6/2011']

[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

2011-01-03 00:00:00


[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

In [15]:
#datetime.strptime是通过已知格式进行日期解析的最佳方式。
#但是每次都要编写格式定义是很麻烦的事情，尤其是对于一些常见
#的日期格式。这种情况下，你可以用dateutil这个第三方包中的
#parser.parse方法（pandas中已经自动安装好了）
from dateutil.parser import parse

parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [16]:
#dateutil可以解析几乎所有人类能够理解的日期表示形式
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

In [17]:
#在国际通用的格式中，日出现在月的前面很普遍，
#传入dayfirst=True即可解决这个问题
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

In [20]:
#pandas通常是用于处理成组日期的，不管这些日期是DataFrame的轴索引
#还是列。to_datetime方法可以解析多种不同的日期表示形式。对标准日
#期格式（如ISO8601）的解析非常快
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

In [21]:
#它还可以处理缺失值（None、空字符串等）
idx = pd.to_datetime(datestrs + [None])

idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [22]:
idx[2]

NaT

In [23]:
pd.isnull(idx)
#NaT（Not a Time）是pandas中时间戳数据的null值
#注意：dateutil.parser是一个实用但不完美的工具。
#比如说，它会把一些原本不是日期的字符串认作是日期
#（比如"42"会被解析为2042年的今天）

array([False, False,  True], dtype=bool)

### 11.2 时间序列基础

In [24]:
#pandas最基本的时间序列类型就是以时间戳
#（通常以Python字符串或datatime对象表示）
#为索引的Series
from datetime import datetime

dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.randn(6), index=dates)

ts

2011-01-02   -0.078437
2011-01-05   -0.205302
2011-01-07    0.597924
2011-01-08   -0.818515
2011-01-10    0.132945
2011-01-12    0.148380
dtype: float64

In [25]:
#这些datetime对象实际上是被放在一个DatetimeIndex中的
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [27]:
ts[::2]

2011-01-02   -0.078437
2011-01-07    0.597924
2011-01-10    0.132945
dtype: float64

In [26]:
#跟其他Series一样，不同索引的时间序列之间的算术运算会自动按日期对齐
ts + ts[::2]

2011-01-02   -0.156874
2011-01-05         NaN
2011-01-07    1.195848
2011-01-08         NaN
2011-01-10    0.265889
2011-01-12         NaN
dtype: float64

In [28]:
#pandas用NumPy的datetime64数据类型以纳秒形式存储时间戳
ts.index.dtype

dtype('<M8[ns]')

In [29]:
#DatetimeIndex中的各个标量值是pandas的Timestamp对象
stamp = ts.index[0]

stamp
#只要有需要，TimeStamp可以随时自动转换为datetime对象。
#此外，它还可以存储频率信息（如果有的话），且知道如何执行时
#区转换以及其他操作。稍后将对此进行详细讲解。

Timestamp('2011-01-02 00:00:00')

#### 索引、选取、子集构造

In [31]:
#当你根据标签索引选取数据时，时间序列和其它的pandas.Series很像
stamp = ts.index[2]

ts[stamp]

0.59792375376579832

In [32]:
#还有一种更为方便的用法：传入一个可以被解释为日期的字符串
print(ts['1/10/2011'])
print(ts['20110110'])

0.132944608909
0.132944608909


In [33]:
#对于较长的时间序列，只需传入“年”或“年月”即可轻松选取数据的切片
longer_ts = pd.Series(np.random.randn(1000),
                      index=pd.date_range('1/1/2000', periods=1000))

print(longer_ts)

print(longer_ts['2001'])

2000-01-01   -0.780320
2000-01-02   -0.157200
2000-01-03    0.710261
2000-01-04    0.757090
2000-01-05    1.072024
2000-01-06    2.317108
2000-01-07    1.802059
2000-01-08   -0.231025
2000-01-09   -1.079804
2000-01-10   -0.122899
2000-01-11   -0.354657
2000-01-12    1.783776
2000-01-13    1.118190
2000-01-14    0.737508
2000-01-15    0.808868
2000-01-16   -1.281741
2000-01-17    0.030669
2000-01-18   -0.036913
2000-01-19   -0.829042
2000-01-20   -0.915724
2000-01-21    0.787562
2000-01-22   -0.360872
2000-01-23   -0.359229
2000-01-24    0.958767
2000-01-25   -0.383858
2000-01-26    0.993436
2000-01-27    0.594636
2000-01-28   -0.748152
2000-01-29   -0.610141
2000-01-30   -0.575233
                ...   
2002-08-28    0.238982
2002-08-29   -0.144295
2002-08-30   -0.392323
2002-08-31   -0.530434
2002-09-01    0.987275
2002-09-02   -0.342543
2002-09-03   -0.202257
2002-09-04    0.542156
2002-09-05   -0.896534
2002-09-06    0.798716
2002-09-07   -0.161535
2002-09-08   -1.112349
2002-09-09 

In [34]:
#这里，字符串“2001”被解释成年，并根据它选取时间区间。指定月也同样奏效
longer_ts['2001-05']

2001-05-01    0.430495
2001-05-02   -0.236329
2001-05-03   -0.826956
2001-05-04   -0.131613
2001-05-05   -1.286526
2001-05-06   -0.012823
2001-05-07    0.261024
2001-05-08    0.515479
2001-05-09   -1.473017
2001-05-10   -1.274623
2001-05-11   -0.195775
2001-05-12    0.234231
2001-05-13   -1.676276
2001-05-14   -1.151441
2001-05-15   -1.540982
2001-05-16    0.266079
2001-05-17    1.082440
2001-05-18    0.196095
2001-05-19    0.904141
2001-05-20   -1.671861
2001-05-21   -0.722376
2001-05-22   -0.465751
2001-05-23   -0.395678
2001-05-24    1.297827
2001-05-25   -1.011106
2001-05-26   -0.104827
2001-05-27    0.210601
2001-05-28    0.150679
2001-05-29    1.044160
2001-05-30   -0.529383
2001-05-31   -1.766727
Freq: D, dtype: float64

In [35]:
#datetime对象也可以进行切片
ts[datetime(2011, 1, 7):]

2011-01-07    0.597924
2011-01-08   -0.818515
2011-01-10    0.132945
2011-01-12    0.148380
dtype: float64

In [36]:
#由于大部分时间序列数据都是按照时间先后排序的，
#因此你也可以用不存在于该时间序列中的时间戳对其进行切片（即范围查询）
print(ts)
print(ts['1/6/2011':'1/11/2011'])

2011-01-02   -0.078437
2011-01-05   -0.205302
2011-01-07    0.597924
2011-01-08   -0.818515
2011-01-10    0.132945
2011-01-12    0.148380
dtype: float64
2011-01-07    0.597924
2011-01-08   -0.818515
2011-01-10    0.132945
dtype: float64


In [40]:
#跟之前一样，你可以传入字符串日期、datetime或Timestamp。
#注意，这样切片所产生的是原时间序列的视图，跟NumPy数组的切片
#运算是一样的。
#这意味着，没有数据被复制，对切片进行修改会反映到原始数据上。
#此外，还有一个等价的实例方法也可以截取两个日期之间TimeSeries
ts.truncate(after='1/9/2011')

2011-01-02   -0.078437
2011-01-05   -0.205302
2011-01-07    0.597924
2011-01-08   -0.818515
dtype: float64

In [41]:
#面这些操作对DataFrame也有效。例如，对DataFrame的行进行索引
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')

long_df = pd.DataFrame(np.random.randn(100, 4),
                      index=dates,
                      columns=['Colorado', 'Texas',
                               'New York','Ohio'])

long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,0.860279,-1.009243,1.158264,-1.054207
2001-05-09,-0.747229,-1.119753,0.300877,0.680670
2001-05-16,0.763386,-0.366488,-0.130694,0.399849
2001-05-23,0.559284,1.239230,-1.196260,-0.145100
2001-05-30,0.986520,0.851139,-0.268293,-0.521609


#### 带有重复索引的时间序列

In [43]:
#在某些应用场景中，可能会存在多个观测数据落在同一个时间点上的情况。
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)

dup_ts


2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [44]:
dup_ts.index.is_unique

False

In [45]:
#对这个时间序列进行索引，要么产生标量值，要么产生切片，
#具体要看所选的时间点是否重复
print(dup_ts['1/3/2000'])

dup_ts['1/2/2000']

4


2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

In [46]:
#假设你想要对具有非唯一时间戳的数据进行聚合。
#一个办法是使用groupby，并传入level=0
grouped = dup_ts.groupby(level=0)
print(grouped.mean())
grouped.count()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64


2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

### 11.3 日期的范围、频率
pandas中的原生时间序列一般被认为是不规则的，也就是说，它们没有<br/>
固定的频率。对于大部分应用程序而言，这是无所谓的。但是，它常常<br/>需要以某种相对固定的频率进行分析，比如每日、每月、每15分钟等<br/>（这样自然会在时间序列中引入缺失值）。幸运的是，pandas有<br/>一整套标准时间序列频率以及用于重采样、频率推断、生成固定频率日<br/>期范围的工具。例如，我们可以将之前那个时间序列转换为一个具有固<br/>定频率（每日）的时间序列，只需调用resample即可

In [51]:
print(ts)
resampler = ts.resample('D')
#字符串“D”是每天的意思


2011-01-02   -0.078437
2011-01-05   -0.205302
2011-01-07    0.597924
2011-01-08   -0.818515
2011-01-10    0.132945
2011-01-12    0.148380
dtype: float64


#### 生成日期范围

In [48]:
#虽然我之前用的时候没有明说，但你可能已经猜到pandas.date_range可
#用于根据指定的频率生成指定长度的DatetimeIndex
index = pd.date_range('2012-04-01','2012-06-01')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [52]:
#默认情况下，date_range会产生按天计算的时间点。如果只传入起始或结束日期，
#那就还得传入一个表示一段时间的数字
print(pd.date_range(start='2012-04-01', periods=20))

pd.date_range(end='2012-06-01', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')


DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [53]:
#起始和结束日期定义了日期索引的严格边界。例如，如果你想要生成一个由
#每月最后一个工作日组成的日期索引，可以传入"BM"频率（
#表示business end of month，表11-4是频率列表），
#这样就只会包含时间间隔内（或刚好在边界上的）符合频率要求的日期
pd.date_range('2000-01-01', '2000-12-01', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [54]:
pd.date_range('2012-05-02 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [55]:
#有时，虽然起始和结束日期带有时间信息，但你希望产生一组被规范
#化（normalize）到午夜的时间戳。normalize选项即可实现该功能
pd.date_range('2012-05-02 12:56:31', periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

#### 频率和日期偏移量

In [56]:
#pandas中的频率是由一个基础频率（base frequency）和一个乘数组
#成的。基础频率通常以一个字符串别名表示，比如"M"表示每月，"H"表示
#每小时。对于每个基础频率，都有一个被称为日期偏移量（date offset）
#的对象与之对应。例如，按小时计算的频率可以用Hour类表示
from pandas.tseries.offsets import Hour, Minute
hour = Hour()
hour

<Hour>

In [58]:
#传入一个整数即可定义偏移量的倍数
four_hours = Hour(4)
four_hours

<4 * Hours>

In [59]:
#一般来说，无需明确创建这样的对象，只需使用诸如"H"或"4H"这样的字符串别名即可。
#在基础频率前面放上一个整数即可创建倍数
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [60]:
Hour(2) + Minute(30)
#大部分偏移量对象都可通过加法进行连接

<150 * Minutes>

In [61]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')
#同理，你也可以传入频率字符串（如"2h30min"），这种字符串可以被高效地
#解析为等效的表达式

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

#### WOM日期

In [62]:
#WOM（Week Of Month）是一种非常实用的频率类，它以WOM开头。
#它使你能获得诸如“每月第3个星期五”之类的日期
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')

list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

#### 移动（超前和滞后）数据

In [64]:
#移动（shifting）指的是沿着时间轴将数据前移或后移。
#Series和DataFrame都有一个shift方法用于执行单纯
#的前移或后移操作，保持索引不变
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2000',periods=4, freq='M'))
    
print(ts)
print(ts.shift(2))
print(ts.shift(-2))
#当我们这样进行移动时，就会在时间序列的前面或后面产生缺失数据

2000-01-31   -0.012011
2000-02-29    0.921009
2000-03-31    0.220786
2000-04-30    2.077673
Freq: M, dtype: float64
2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.012011
2000-04-30    0.921009
Freq: M, dtype: float64
2000-01-31    0.220786
2000-02-29    2.077673
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64


In [65]:
#shift通常用于计算一个时间序列或多个时间序列（如DataFrame的列）中的百分比变化。
#可以这样表达
ts / ts.shift(1) - 1

2000-01-31          NaN
2000-02-29   -77.678747
2000-03-31    -0.760278
2000-04-30     8.410362
Freq: M, dtype: float64

In [66]:
#由于单纯的移位操作不会修改索引，所以部分数据会被丢弃。
#因此，如果频率已知，则可以将其传给shift以便实现对时间
#戳进行位移而不是对数据进行简单位移
ts.shift(2, freq='M')

2000-03-31   -0.012011
2000-04-30    0.921009
2000-05-31    0.220786
2000-06-30    2.077673
Freq: M, dtype: float64

In [67]:
#这里还可以使用其他频率，于是你就能非常灵活地对数据进行超前和滞后处理了
print(ts.shift(3, freq='D'))

ts.shift(1, freq='90T')

2000-02-03   -0.012011
2000-03-03    0.921009
2000-04-03    0.220786
2000-05-03    2.077673
dtype: float64


2000-01-31 01:30:00   -0.012011
2000-02-29 01:30:00    0.921009
2000-03-31 01:30:00    0.220786
2000-04-30 01:30:00    2.077673
Freq: M, dtype: float64

#### 通过偏移量对日期进行位移

In [68]:
#pandas的日期偏移量还可以用在datetime或Timestamp对象上
from pandas.tseries.offsets import Day, MonthEnd

now = datetime(2011, 11, 17)

now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

In [69]:
#如果加的是锚点偏移量（比如MonthEnd）
#，第一次增量会将原日期向前滚动到符合频率规则的下一个日期
print(now + MonthEnd())

now + MonthEnd(2)

2011-11-30 00:00:00


Timestamp('2011-12-31 00:00:00')

In [70]:
#通过锚点偏移量的rollforward和rollback方法，
#可明确地将日期向前或向后“滚动”
offset = MonthEnd()

print(offset.rollforward(now))

print(offset.rollback(now))

2011-11-30 00:00:00
2011-10-31 00:00:00


In [71]:
#日期偏移量还有一个巧妙的用法，即结合groupby使用这两个“滚动”方法
ts = pd.Series(np.random.randn(20),
               index=pd.date_range('1/15/2000', periods=20, freq='4d'))
print(ts)
ts.groupby(offset.rollforward).mean()

2000-01-15   -0.094565
2000-01-19   -1.835206
2000-01-23    1.765151
2000-01-27    0.187591
2000-01-31   -0.680474
2000-02-04   -0.571294
2000-02-08    0.061240
2000-02-12    1.097122
2000-02-16    0.027283
2000-02-20   -0.950635
2000-02-24    0.259812
2000-02-28   -0.358216
2000-03-03   -0.836609
2000-03-07    0.122124
2000-03-11    2.218644
2000-03-15    0.837074
2000-03-19    0.075851
2000-03-23   -1.166614
2000-03-27   -2.218385
2000-03-31    0.495042
Freq: 4D, dtype: float64


2000-01-31   -0.131500
2000-02-29   -0.062098
2000-03-31   -0.059109
dtype: float64

In [72]:
#当然，更简单、更快速地实现该功能的办法是使用resample
ts.resample('M').mean()

2000-01-31   -0.131500
2000-02-29   -0.062098
2000-03-31   -0.059109
Freq: M, dtype: float64

### 11.4 时区处理

In [74]:
import pytz

pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [75]:
tz = pytz.timezone('America/New_York')

tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

#### 时区本地化和转换

In [76]:
#默认情况下，pandas中的时间序列是单纯（naive）的时区。
#看看下面这个时间序列
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-09 09:30:00    0.855309
2012-03-10 09:30:00   -0.450124
2012-03-11 09:30:00   -0.600115
2012-03-12 09:30:00   -1.689504
2012-03-13 09:30:00   -1.524979
2012-03-14 09:30:00   -0.900068
Freq: D, dtype: float64

In [77]:
print(ts.index.tz)

None


In [78]:
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')


DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [79]:
print(ts)
ts_utc = ts.tz_localize('UTC')
print(ts_utc)
print(ts_utc.index)

2012-03-09 09:30:00    0.855309
2012-03-10 09:30:00   -0.450124
2012-03-11 09:30:00   -0.600115
2012-03-12 09:30:00   -1.689504
2012-03-13 09:30:00   -1.524979
2012-03-14 09:30:00   -0.900068
Freq: D, dtype: float64
2012-03-09 09:30:00+00:00    0.855309
2012-03-10 09:30:00+00:00   -0.450124
2012-03-11 09:30:00+00:00   -0.600115
2012-03-12 09:30:00+00:00   -1.689504
2012-03-13 09:30:00+00:00   -1.524979
2012-03-14 09:30:00+00:00   -0.900068
Freq: D, dtype: float64
DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')


In [80]:
#一旦时间序列被本地化到某个特定时区，
#就可以用tz_convert将其转换到别的时区了
ts_utc.tz_convert('America/New_York')

2012-03-09 04:30:00-05:00    0.855309
2012-03-10 04:30:00-05:00   -0.450124
2012-03-11 05:30:00-04:00   -0.600115
2012-03-12 05:30:00-04:00   -1.689504
2012-03-13 05:30:00-04:00   -1.524979
2012-03-14 05:30:00-04:00   -0.900068
Freq: D, dtype: float64

In [81]:
#对于上面这种时间序列（它跨越了美国东部时区的夏令时转变期），
#我们可以将其本地化到EST，然后转换为UTC或柏林时间
ts_eastern = ts.tz_localize('America/New_York')
print(ts_eastern.tz_convert('UTC'))
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 14:30:00+00:00    0.855309
2012-03-10 14:30:00+00:00   -0.450124
2012-03-11 13:30:00+00:00   -0.600115
2012-03-12 13:30:00+00:00   -1.689504
2012-03-13 13:30:00+00:00   -1.524979
2012-03-14 13:30:00+00:00   -0.900068
Freq: D, dtype: float64


2012-03-09 15:30:00+01:00    0.855309
2012-03-10 15:30:00+01:00   -0.450124
2012-03-11 14:30:00+01:00   -0.600115
2012-03-12 14:30:00+01:00   -1.689504
2012-03-13 14:30:00+01:00   -1.524979
2012-03-14 14:30:00+01:00   -0.900068
Freq: D, dtype: float64

In [82]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

#### 操作时区意识型Timestamp对象

In [83]:
#跟时间序列和日期范围差不多，独立的Timestamp对象也能被从单纯
#型（naive）本地化为时区意识型（time zone-aware），并从一个
#时区转换到另一个时区
stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('America/New_York')

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

In [84]:
#在创建Timestamp时，还可以传入一个时区信息
stamp_moscow = pd.Timestamp('2011-03-12 04:00', tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

In [85]:
#时区意识型Timestamp对象在内部保存了一个UTC时间戳值
#（自UNIX纪元（1970年1月1日）算起的纳秒数）。这个UTC值在
#时区转换过程中是不会发生变化的
stamp_utc.value
stamp_utc.tz_convert('America/New_York').value

1299902400000000000

In [86]:
#当使用pandas的DateOffset对象执行时间算术运算时，运算过程会
#自动关注是否存在夏令时转变期。这里，我们创建了在DST转变之前的
#时间戳。首先，来看夏令时转变前的30分钟
from pandas.tseries.offsets import Hour
stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')
print(stamp)
stamp+Hour()

2012-03-12 01:30:00-04:00


Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [87]:
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
print(stamp)
stamp+2*Hour()

2012-11-04 00:30:00-04:00


Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

#### 不同时区之间的运算

In [88]:
#如果两个时间序列的时区不同，在将它们合并到一起时，最终结果就会是UTC。
#由于时间戳其实是以UTC存储的，所以这是一个很简单的运算，并不需要发生
#任何转换
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
print(ts)

ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
result = ts1 +ts2
result.index

2012-03-07 09:30:00    0.449086
2012-03-08 09:30:00    0.616402
2012-03-09 09:30:00   -1.262287
2012-03-12 09:30:00    0.287144
2012-03-13 09:30:00    0.493287
2012-03-14 09:30:00   -0.324612
2012-03-15 09:30:00   -0.239376
2012-03-16 09:30:00   -1.361306
2012-03-19 09:30:00   -0.097037
2012-03-20 09:30:00    0.205789
Freq: B, dtype: float64


DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

### 11.5 时期及其算数运算
时期（period）表示的是时间区间，比如数日、数月、数季、数年等。<br/>Period类所表示的就是这种数据类型，其构造函数需要用到一个<br/>字符串或整数，以及表11-4中的频率

In [89]:
p = pd.Period(2007, freq='A-DEC')

p

Period('2007', 'A-DEC')

In [90]:
#这里，这个Period对象表示的是从2007年1月1日到2007年12月31日之间
#的整段时间。只需对Period对象加上或减去一个整数即可达到根据其频率
#进行位移的效果
print(p + 5)
print(p - 2)

2012
2005


In [91]:
#如果两个Period对象拥有相同的频率，则它们的差就是它们之间的单位数量
pd.Period('2014', freq='A-DEC') - p

7

In [92]:
#period_range函数可用于创建规则的时期范围
rng = pd.period_range('2000-01-01', '2000-06-30', freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

In [93]:
#PeriodIndex类保存了一组Period，
#它可以在任何pandas数据结构中被用作轴索引
pd.Series(np.random.randn(6), index=rng)

2000-01    1.197440
2000-02    1.300641
2000-03   -0.458536
2000-04    0.277274
2000-05   -0.968573
2000-06    0.095032
Freq: M, dtype: float64

In [94]:
#如果你有一个字符串数组，你也可以使用PeriodIndex类
values = ['2001Q3', '2002Q2', '2003Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

#### 时期的频率转换

In [95]:
#Period和PeriodIndex对象都可以通过其asfreq方法被转换成别的频率。
#假设我们有一个年度时期，希望将其转换为当年年初或年末的一个月度时期。
#该任务非常简单
p = pd.Period('2007', freq='A-DEC')
print(p)
print(p.asfreq('M', how='start'))
print(p.asfreq('M', how='end'))

2007
2007-01
2007-12


In [96]:
#你可以将Period('2007','A-DEC')看做一个被划分为多个月度时期
#的时间段中的游标。
p = pd.Period('2007', freq='A-JUN')
print(p)
print(p.asfreq('M', 'start'))
print(p.asfreq('M', 'end'))

2007
2006-07
2007-06


In [97]:
#在将高频率转换为低频率时，超时期（superperiod）是由子时期
#（subperiod）所属的位置决定的。例如，在A-JUN频率中，
#月份“2007年8月”实际上是属于周期“2008年”的
p = pd.Period('Aug-2007', 'M')
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

In [98]:
#完整的PeriodIndex或TimeSeries的频率转换方式也是如此
rng = pd.period_range('2006', '2009', freq='A-DEC')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
print(ts)
ts.asfreq('M', how='start')

2006    0.243796
2007   -0.107236
2008   -1.404763
2009   -0.380289
Freq: A-DEC, dtype: float64


2006-01    0.243796
2007-01   -0.107236
2008-01   -1.404763
2009-01   -0.380289
Freq: M, dtype: float64

In [99]:
#这里，根据年度时期的第一个月，每年的时期被取代为每月的时期。
#如果我们想要每年的最后一个工作日，我们可以使用“B”频率，并指明想要该时期的末尾
ts.asfreq('B', how='end')

2006-12-29    0.243796
2007-12-31   -0.107236
2008-12-31   -1.404763
2009-12-31   -0.380289
Freq: B, dtype: float64

#### 按季度计算的时期频率
季度型数据在会计、金融等领域中很常见。许多季度型数据都会涉及“财年<br/>末”的概念，通常是一年12个月中某月的最后一个日历日或工作日。
就这一点来说，时期"2012Q4"根据财年末的不同会有不同的含义。<br />pandas支持12种可能的季度型频率，即Q-JAN到Q-DEC

In [100]:
p = pd.Period('2012Q4', freq='Q-JAN')
p

Period('2012Q4', 'Q-JAN')

In [101]:
#在以1月结束的财年中，2012Q4是从11月到1月
#（将其转换为日型频率就明白了）
print(p.asfreq('D', 'start'))
print(p.asfreq('D', 'end'))

2011-11-01
2012-01-31


In [102]:
#因此，Period之间的算术运算会非常简单。
#例如，要获取该季度倒数第二个工作日下午4点的时间戳，你可以这样
p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
print(p4pm)
p4pm.to_timestamp()

2012-01-30 16:00


Timestamp('2012-01-30 16:00:00')

In [103]:
#period_range可用于生成季度型范围。
#季度型范围的算术运算也跟上面是一样的
rng = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')
ts = pd.Series(np.arange(len(rng)), index=rng)
print(ts)

new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
ts.index = new_rng.to_timestamp()
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int64


2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int64

#### 将Timestamp转换为Period（及其反向过程）

In [104]:
#通过使用to_period方法，可以将由时间戳索引的Series和DataFrame对象转换
#为以时期索引
rng = pd.date_range('2000-01-01', periods=3, freq='M')
ts = pd.Series(np.random.randn(3), index=rng)
print(ts)
pts = ts.to_period()
pts

2000-01-31    0.467888
2000-02-29    2.329749
2000-03-31   -0.767399
Freq: M, dtype: float64


2000-01    0.467888
2000-02    2.329749
2000-03   -0.767399
Freq: M, dtype: float64

In [105]:
#由于时期指的是非重叠时间区间，因此对于给定的频率，
#一个时间戳只能属于一个时期。新PeriodIndex的频率默认是从时间
#戳推断而来的，你也可以指定任何别的频率。结果中允许存在重复时期
rng = pd.date_range('1/29/2000', periods=6, freq='D')

ts2 = pd.Series(np.random.randn(6), index=rng)

print(ts2)

ts2.to_period('M')

2000-01-29   -1.322982
2000-01-30    0.967862
2000-01-31   -0.222105
2000-02-01   -0.302882
2000-02-02   -0.695805
2000-02-03    0.291144
Freq: D, dtype: float64


2000-01   -1.322982
2000-01    0.967862
2000-01   -0.222105
2000-02   -0.302882
2000-02   -0.695805
2000-02    0.291144
Freq: M, dtype: float64

In [107]:
#要转换回时间戳，使用to_timestamp即可
pts = ts2.to_period()

print(pts)

pts.to_timestamp(how='end')

2000-01-29   -1.322982
2000-01-30    0.967862
2000-01-31   -0.222105
2000-02-01   -0.302882
2000-02-02   -0.695805
2000-02-03    0.291144
Freq: D, dtype: float64


2000-01-29   -1.322982
2000-01-30    0.967862
2000-01-31   -0.222105
2000-02-01   -0.302882
2000-02-02   -0.695805
2000-02-03    0.291144
Freq: D, dtype: float64

#### 通过数组创建PeriodIndex
固定频率的数据集通常会将时间信息分开存放在多个列中。<br/>
例如，在下面这个宏观经济数据集中，年度和季度就分别存放在不同的列中

In [108]:
data = pd.read_csv('examples/macrodata.csv')

print(data.head(5))

print(data.year)

print(data.quarter)


     year  quarter   realgdp  realcons  realinv  realgovt  realdpi    cpi  \
0  1959.0      1.0  2710.349    1707.4  286.898   470.045   1886.9  28.98   
1  1959.0      2.0  2778.801    1733.7  310.859   481.301   1919.7  29.15   
2  1959.0      3.0  2775.488    1751.8  289.226   491.260   1916.4  29.35   
3  1959.0      4.0  2785.204    1753.7  299.356   484.052   1931.3  29.37   
4  1960.0      1.0  2847.699    1770.5  331.722   462.199   1955.5  29.54   

      m1  tbilrate  unemp      pop  infl  realint  
0  139.7      2.82    5.8  177.146  0.00     0.00  
1  141.7      3.08    5.1  177.830  2.34     0.74  
2  140.5      3.82    5.3  178.657  2.74     1.09  
3  140.0      4.33    5.6  179.386  0.27     4.06  
4  139.6      3.50    5.2  180.007  2.31     1.19  
0      1959.0
1      1959.0
2      1959.0
3      1959.0
4      1960.0
5      1960.0
6      1960.0
7      1960.0
8      1961.0
9      1961.0
10     1961.0
11     1961.0
12     1962.0
13     1962.0
14     1962.0
15     1962.0
1

In [109]:
#通过将这些数组以及一个频率传入PeriodIndex，
#就可以将它们合并成DataFrame的一个索引
index = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                       freq='Q-DEC')

index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')

In [110]:
data.index = index

data.infl

1959Q1    0.00
1959Q2    2.34
1959Q3    2.74
1959Q4    0.27
1960Q1    2.31
1960Q2    0.14
1960Q3    2.70
1960Q4    1.21
1961Q1   -0.40
1961Q2    1.47
1961Q3    0.80
1961Q4    0.80
1962Q1    2.26
1962Q2    0.13
1962Q3    2.11
1962Q4    0.79
1963Q1    0.53
1963Q2    2.75
1963Q3    0.78
1963Q4    2.46
1964Q1    0.13
1964Q2    0.90
1964Q3    1.29
1964Q4    2.05
1965Q1    1.28
1965Q2    2.54
1965Q3    0.89
1965Q4    2.90
1966Q1    4.99
1966Q2    2.10
          ... 
2002Q2    1.56
2002Q3    2.66
2002Q4    3.08
2003Q1    1.31
2003Q2    1.09
2003Q3    2.60
2003Q4    3.02
2004Q1    2.35
2004Q2    3.61
2004Q3    3.58
2004Q4    2.09
2005Q1    4.15
2005Q2    1.85
2005Q3    9.14
2005Q4    0.40
2006Q1    2.60
2006Q2    3.97
2006Q3   -1.58
2006Q4    3.30
2007Q1    4.58
2007Q2    2.75
2007Q3    3.45
2007Q4    6.38
2008Q1    2.82
2008Q2    8.53
2008Q3   -3.16
2008Q4   -8.79
2009Q1    0.94
2009Q2    3.37
2009Q3    3.56
Freq: Q-DEC, Name: infl, Length: 203, dtype: float64

### 11.6 重采样及频率转换
重采样（resampling）指的是将时间序列从一个频率转换到另一个频率<br/>的处理过程。将高频率数据聚合到低频率称为降采样<br/>（downsampling），而将低频率数据转换到高频率则称为升采样<br/>（upsampling）。并不是所有的重采样都能被划分到这两个大类中。<br/>例如，将W-WED（每周三）转换为W-FRI既不是降采样也不是升采样

In [111]:
#pandas对象都带有一个resample方法，它是各种频率转换工作的主力函数。
#resample有一个类似于groupby的API，调用resample可以分组数据，
#然后会调用一个聚合函数
rng = pd.date_range('2000-01-01', periods=100, freq='D')

ts = pd.Series(np.random.randn(len(rng)), index=rng)

print(ts)

print(ts.resample('M').mean())
print(ts.resample('M', kind='period').mean())

2000-01-01   -0.536279
2000-01-02    0.410834
2000-01-03   -0.445233
2000-01-04   -0.486858
2000-01-05    1.150901
2000-01-06    0.867141
2000-01-07   -0.469346
2000-01-08   -0.454937
2000-01-09    1.202657
2000-01-10    1.192396
2000-01-11    0.422508
2000-01-12    0.430507
2000-01-13    0.100297
2000-01-14    0.935103
2000-01-15   -0.428412
2000-01-16    0.342369
2000-01-17   -0.629025
2000-01-18   -2.134723
2000-01-19    1.439643
2000-01-20   -0.716543
2000-01-21   -1.327908
2000-01-22   -0.035384
2000-01-23    0.741481
2000-01-24   -1.345660
2000-01-25    1.336838
2000-01-26   -0.311095
2000-01-27   -1.282048
2000-01-28   -0.564690
2000-01-29   -0.885918
2000-01-30    0.370331
                ...   
2000-03-11    2.177407
2000-03-12    1.018033
2000-03-13    0.758339
2000-03-14    1.929953
2000-03-15    0.648688
2000-03-16   -1.367605
2000-03-17    0.258108
2000-03-18    0.685376
2000-03-19    1.101585
2000-03-20   -0.160887
2000-03-21    1.815267
2000-03-22   -0.674339
2000-03-23 

#### 降采样
将数据聚合到规律的低频率是一件非常普通的时间序列处理任务。待聚合<br/>的数据不必拥有固定的频率，期望的频率会自动定义聚合的面元边界，<br/>这些面元用于将时间序列拆分为多个片段。例如，要转换到月度频率<br/>（'M'或'BM'），数据需要被划分到多个单月时间段中。各时间段都是<br/>半开放的。一个数据点只能属于一个时间段，所有时间段的并集必须能<br/>组成整个时间帧。在用resample对数据进行降采样时，需要考虑两样<br/>东西：
<br/>各区间哪边是闭合的。
<br/>如何标记各个聚合面元，用区间的开头还是末尾。

In [112]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')
ts = pd.Series(np.arange(12), index=rng)
ts

#传入的频率将会以“5分钟”的增量定义面元边界。默认情况下，
#面元的右边界是包含的，因此00:00到00:05的区间中是包含00:05的。
#传入closed='left'会让区间以左边界闭合

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int64

In [113]:
ts.resample('5min', closed='right').sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int64

In [114]:
ts.resample('5min', closed='left').sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int64

In [115]:
#如你所见，最终的时间序列是以各面元右边界的时间戳进行标记的。
#传入label='right'即可用面元的邮编界对其进行标记
ts.resample('5min', closed='right', label='right').sum()

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5T, dtype: int64

In [116]:
#最后，你可能希望对结果索引做一些位移，比如从右边界减去一秒以便
#更容易明白该时间戳到底表示的是哪个区间。只需通过loffset设置一个
#字符串或日期偏移量即可实现这个目的
ts.resample('5min', closed='right',
            label='right', loffset='-1s').sum()

1999-12-31 23:59:59     0
2000-01-01 00:04:59    15
2000-01-01 00:09:59    40
2000-01-01 00:14:59    11
Freq: 5T, dtype: int64

In [117]:
ts.resample('5min', closed='right',
            label='right', loffset='-1s').sum()

1999-12-31 23:59:59     0
2000-01-01 00:04:59    15
2000-01-01 00:09:59    40
2000-01-01 00:14:59    11
Freq: 5T, dtype: int64

#### OHLC重采样
金融领域中有一种无所不在的时间序列聚合方式，即计算各面元的四个值：<br/>第一个值（open，开盘）、最后一个值（close，收盘）、最大值<br/>（high，最高）以及最小值（low，最低）。传入how='ohlc'即可得<br/>到一个含有这四种聚合值的DataFrame。整个过程很高效，只需一次扫<br/>描即可计算出结果

In [118]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


#### 升采样和插值

In [119]:
#在将数据从低频率转换到高频率时，就不需要聚合了。
#我们来看一个带有一些周型数据的DataFrame
frame = pd.DataFrame(np.random.randn(2, 4),
                      index=pd.date_range('1/1/2000', periods=2,
                                          freq='W-WED'),
                      columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,0.339006,-0.679871,-0.745304,0.993155
2000-01-12,0.121356,-0.275438,-0.287654,-0.641007


In [120]:
#当你对这个数据进行聚合，每组只有一个值，这样就会引入缺失值。
#我们使用asfreq方法转换成高频，不经过聚合
df_daily = frame.resample('D').asfreq()

df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,0.339006,-0.679871,-0.745304,0.993155
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,0.121356,-0.275438,-0.287654,-0.641007


In [121]:
#假设你想要用前面的周型值填充“非星期三”。
#resampling的填充和插值方式跟fillna和reindex的一样
frame.resample('D').ffill()

,Colorado,Texas,New York,Ohio
2000-01-05,0.339006,-0.679871,-0.745304,0.993155
2000-01-06,0.339006,-0.679871,-0.745304,0.993155
2000-01-07,0.339006,-0.679871,-0.745304,0.993155
2000-01-08,0.339006,-0.679871,-0.745304,0.993155
2000-01-09,0.339006,-0.679871,-0.745304,0.993155
2000-01-10,0.339006,-0.679871,-0.745304,0.993155
2000-01-11,0.339006,-0.679871,-0.745304,0.993155
2000-01-12,0.121356,-0.275438,-0.287654,-0.641007


In [122]:
#同样，这里也可以只填充指定的时期数（
#目的是限制前面的观测值的持续使用距离）
frame.resample('D').ffill(limit=2)

,Colorado,Texas,New York,Ohio
2000-01-05,0.339006,-0.679871,-0.745304,0.993155
2000-01-06,0.339006,-0.679871,-0.745304,0.993155
2000-01-07,0.339006,-0.679871,-0.745304,0.993155
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,0.121356,-0.275438,-0.287654,-0.641007


In [123]:
#注意，新的日期索引完全没必要跟旧的重叠
frame.resample('W-THU').ffill()

,Colorado,Texas,New York,Ohio
2000-01-06,0.339006,-0.679871,-0.745304,0.993155
2000-01-13,0.121356,-0.275438,-0.287654,-0.641007


#### 通过时期进行重采样

In [124]:
frame = pd.DataFrame(np.random.randn(24, 4),
                     index=pd.period_range('1-2000', '12-2001',
                                           freq='M'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])

print(frame[:5])

annual_frame = frame.resample('A-DEC').mean()

annual_frame

         Colorado     Texas  New York      Ohio
2000-01 -1.676186  0.372070  0.339171  1.007348
2000-02 -0.619515  0.809665  1.090588 -0.918812
2000-03  0.574990 -1.098432 -0.009892 -0.273628
2000-04  1.577879  1.637728 -0.250668  0.660206
2000-05 -0.789503  1.405938  0.247740 -0.876650


,Colorado,Texas,New York,Ohio
2000,-0.612388,0.444400,-0.180422,-0.185416
2001,-0.485567,-0.173339,-0.196722,0.157755


In [125]:
#升采样要稍微麻烦一些，因为你必须决定在新频率中各区间的哪端用于放置原来的值，
#就像asfreq方法那样。convention参数默认为'start'，也可设置为'end'
print(annual_frame.resample('Q-DEC').ffill())
print(annual_frame.resample('Q-DEC', convention='end').ffill())

        Colorado     Texas  New York      Ohio
2000Q1 -0.612388  0.444400 -0.180422 -0.185416
2000Q2 -0.612388  0.444400 -0.180422 -0.185416
2000Q3 -0.612388  0.444400 -0.180422 -0.185416
2000Q4 -0.612388  0.444400 -0.180422 -0.185416
2001Q1 -0.485567 -0.173339 -0.196722  0.157755
2001Q2 -0.485567 -0.173339 -0.196722  0.157755
2001Q3 -0.485567 -0.173339 -0.196722  0.157755
2001Q4 -0.485567 -0.173339 -0.196722  0.157755
        Colorado     Texas  New York      Ohio
2000Q4 -0.612388  0.444400 -0.180422 -0.185416
2001Q1 -0.612388  0.444400 -0.180422 -0.185416
2001Q2 -0.612388  0.444400 -0.180422 -0.185416
2001Q3 -0.612388  0.444400 -0.180422 -0.185416
2001Q4 -0.485567 -0.173339 -0.196722  0.157755


In [126]:
#由于时期指的是时间区间，所以升采样和降采样的规则就比较严格：
#在降采样中，目标频率必须是源频率的子时期（subperiod）。
#在升采样中，目标频率必须是源频率的超时期（superperiod）。
#如果不满足这些条件，就会引发异常。这主要影响的是按季、年、周计算的
#频率。例如，由Q-MAR定义的时间区间只能升采样为A-MAR、A-JUN、
#A-SEP、A-DEC等
annual_frame.resample('Q-MAR').ffill()

,Colorado,Texas,New York,Ohio
2000Q4,-0.612388,0.444400,-0.180422,-0.185416
2001Q1,-0.612388,0.444400,-0.180422,-0.185416
2001Q2,-0.612388,0.444400,-0.180422,-0.185416
2001Q3,-0.612388,0.444400,-0.180422,-0.185416
2001Q4,-0.485567,-0.173339,-0.196722,0.157755
2002Q1,-0.485567,-0.173339,-0.196722,0.157755
2002Q2,-0.485567,-0.173339,-0.196722,0.157755
2002Q3,-0.485567,-0.173339,-0.196722,0.157755


### 11.7 移动窗口函数
在移动窗口（可以带有指数衰减权数）上计算的各种统计函数也是一类常见于<br/>时间序列的数组变换。这样可以圆滑噪音数据或断裂数据。我将它们称<br/>为移动窗口函数（moving window function），其中还包括那些窗<br/>口不定长的函数（如指数加权移动平均）。跟其他统计函数一样，移动<br/>窗口函数也会自动排除缺失值

In [127]:
close_px_all = pd.read_csv('examples/stock_px_2.csv',
                           parse_dates=True, index_col=0)

close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]

close_px = close_px.resample('B').ffill()

In [128]:
print(close_px.AAPL.plot())

close_px.AAPL.rolling(250).mean().plot()

AxesSubplot(0.125,0.125;0.775x0.755)


In [129]:
appl_std250 = close_px.AAPL.rolling(250, min_periods=10).std()
print(appl_std250[5:12])
appl_std250.plot()

2003-01-09         NaN
2003-01-10         NaN
2003-01-13         NaN
2003-01-14         NaN
2003-01-15    0.077496
2003-01-16    0.074760
2003-01-17    0.112368
Freq: B, Name: AAPL, dtype: float64


In [130]:
expanding_mean = appl_std250.expanding().mean()

In [131]:
close_px.rolling(60).mean().plot(logy=True)

In [132]:
close_px.rolling('20D').mean()

,AAPL,MSFT,XOM
2003-01-02,7.400000,21.110000,29.220000
2003-01-03,7.425000,21.125000,29.230000
2003-01-06,7.433333,21.256667,29.473333
2003-01-07,7.432500,21.425000,29.342500
2003-01-08,7.402000,21.402000,29.240000
2003-01-09,7.391667,21.490000,29.273333
2003-01-10,7.387143,21.558571,29.238571
2003-01-13,7.378750,21.633750,29.197500
2003-01-14,7.370000,21.717778,29.194444
2003-01-15,7.355000,21.757000,29.152000


#### 指数加权函数

In [134]:
import matplotlib.pyplot as plt

In [135]:
aapl_px = close_px.AAPL['2006':'2007']
ma60 = aapl_px.rolling(30, min_periods=20).mean()
ewma60 = aapl_px.ewm(span=30).mean()
print(ma60.plot(style='k--', label='Simple MA'))
print(ewma60.plot(style='k-', label='EW MA'))
plt.legend()

AxesSubplot(0.125,0.125;0.775x0.755)
AxesSubplot(0.125,0.125;0.775x0.755)


#### 二元移动窗口函数

In [136]:
spx_px = close_px_all['SPX']

spx_rets = spx_px.pct_change()

returns = close_px.pct_change()

In [137]:
corr = returns.AAPL.rolling(125, min_periods=100).corr(spx_rets)

corr.plot()

In [138]:
corr = returns.rolling(125, min_periods=100).corr(spx_rets)

corr.plot()

#### 用户定义的移动窗口函数

In [139]:
from scipy.stats import percentileofscore

score_at_2percent = lambda x: percentileofscore(x, 0.02)

result = returns.AAPL.rolling(250).apply(score_at_2percent)

result.plot()

### 11.8 总结